### Step 1: Install Necessary Libraries
First, we need to install the required Python libraries, RDKit and PySoftK. This installation cell handles the process by cloning the pysoftk repository and then using `pip install .` to install the package and all its dependencies as defined in its `setup.py` file.

In [ ]:
# Clone the pysoftk repository from GitHub
!git clone https://github.com/alejandrosantanabonilla/pysoftk

# Navigate to the pysoftk directory
%cd pysoftk

# Install the package from the current directory, which reads from setup.py
!pip install .

### Conformer Generation with Open Babel's Confab Approach

This notebook provides a tutorial on generating molecular conformers using the **Confab** approach from **Open Babel**, as implemented through the **PySoftK** library. We'll briefly explain the method, then run the Python script, and finally visualize the results using **RDKit**.

#### 1. The Confab Method Explained

The **Confab** method is a conformer generation algorithm that aims to produce a diverse set of low-energy 3D molecular structures. It's more efficient than purely random searches because it uses a pre-calculated library of torsion angles, derived from known crystal structures, to guide its exploration. 

The process works in three main steps:
1.  **Systematic Torsion Search**: It identifies the rotatable bonds in a molecule and systematically samples their torsion angles using values from a pre-defined library. This helps in quickly finding chemically relevant conformations.
2.  **Energy Minimization**: Each sampled conformation is subjected to a quick energy minimization using a force field (like **MMFF94**). This step relieves steric strain and ensures that the generated structures are energetically plausible.
3.  **Filtering**: The final set of conformers is filtered based on two criteria to ensure diversity and relevance:
    * **RMSD (Root Mean Square Deviation)**: Redundant conformers (those with an **RMSD** below a certain threshold) are discarded.
    * **Energy Gap**: Only conformers within a specified energy range relative to the lowest-energy one are retained.

#### 2. Python Script for Conformer Generation

The following script uses the **PySoftK** library, which provides a convenient wrapper for **Open Babel's** **Confab** functionality. We'll generate conformers for a triphenylmethane-like molecule.

In [ ]:
# 1. Import necessary libraries.
from rdkit import Chem
from rdkit.Chem import AllChem
from pysoftk.torsional.mol_conformer import ConformerGenerator

# 2. Initialize the ConformerGenerator class.
conf_generator = ConformerGenerator()

# 3. Define the molecule input (SMILES string).
smiles_string = 'c1ccc(-c2ccc(-c3ccc(-c4ccccc4)cc3)cc2)cc1'

# 4. Define the output directory.
output_directory = 'biphenyl_conformers_confab'

# 5. Call the method to generate and save the conformers.
#    nconfs: The maximum number of conformers to generate.
#    rmsd: The RMSD threshold for filtering redundant conformers.
#    energy_gap: The energy gap (in kJ/mol) relative to the minimum energy conformer.
num_conformers_generated = conf_generator.confab_generate_conformers(
    molecule_input=smiles_string,
    output_dir=output_directory,
    base_filename='bi_phenyl',
    nconfs=50,
    rmsd=0.5,
    energy_gap=20.0,
    verbose=True
)

# 6. Check the result.
if num_conformers_generated > 0:
    print(f"\n✅ Conformer generation successful. All files are in '{output_directory}'.")
else:
    print("\n❌ Conformer generation failed. Please check the console output.")

#### 3. Visualizing the Generated Conformers

We will now load the generated `.xyz` files and visualize them using **RDKit**. The previous version had an issue where the image might not display in some environments. We've fixed this by explicitly using `IPython.display.display` to ensure the image is rendered correctly.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import MolsToGridImage
import os
from IPython.display import display

# 1. Load all conformer files from the output directory.
conformers = []
output_directory = 'biphenyl_conformers_confab'
files = sorted([f for f in os.listdir(output_directory) if f.endswith('.xyz')])

for file in files:
    mol = Chem.MolFromXYZFile(os.path.join(output_directory, file))
    if mol:
        # The XYZ file format does not contain bond information, so we need to add it.
        # We'll create a molecule from the SMILES string to get the bonding information
        # and transfer the 3D coordinates from the loaded conformer.
        smiles = 'c1ccc(-c2ccc(-c3ccc(-c4ccccc4)cc3)cc2)cc1'
        mol_with_bonds = Chem.MolFromSmiles(smiles)
        mol_with_bonds = Chem.AddHs(mol_with_bonds)
        
        # Create a new conformer for the molecule with bonds
        new_conf = Chem.Conformer(mol.GetConformer())
        mol_with_bonds.AddConformer(new_conf, assignId=True)
        conformers.append(mol_with_bonds)

print(f"Loaded {len(conformers)} conformers from the directory.")

# 2. Align the conformers for better visual comparison.
if conformers:
    ref_mol = conformers[0]
    for i in range(1, len(conformers)):
        AllChem.AlignMol(conformers[i], ref_mol)

# 3. Display the conformers in a grid.
#    We will display a maximum of 10 conformers for clarity.
conformers_to_display = conformers[:10]
legends = [f'Conf_{i}' for i in range(len(conformers_to_display))]

if conformers_to_display:
    img = MolsToGridImage(conformers_to_display, molsPerRow=5, subImgSize=(200, 200), legends=legends)
    display(img)
else:
    print("No conformers to display.")